In [1]:
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install datasets[audio]


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: datasets[audio]


In [2]:
!module load gcc/8.2.0 python_gpu/3.10.4 r/4.0.2 git-lfs/2.3.0 eth_proxy npm/6.14.9 libsndfile/1.0.23

zsh:1: command not found: module


In [3]:
import torch
from datasets import load_dataset
from datasets import DatasetDict
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import StratifiedKFold


/home/fgirlanda/CIL_sentiment_analysis/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# First make the kfold object
folds = StratifiedKFold(n_splits=5)

#create dataframes for train and test
train_df = pd.read_csv('preprocessed/train_full.csv')
#drop nan
train_df.dropna(inplace=True)
print(train_df.info())
folds_datasets = []
#get folds
for fold_, (train_index, test_index) in enumerate(folds.split(train_df, train_df['label'])):
    train_fold = train_df.iloc[train_index]
    test_fold = train_df.iloc[test_index]
    #take 1% of the train data
    train_fold = train_fold.sample(frac=0.01, random_state=1)
    test_fold = test_fold.sample(frac=0.01, random_state=1)
    dataset_fold = DatasetDict({'train': Dataset.from_pandas(train_fold), 'test': Dataset.from_pandas(test_fold)})
    folds_datasets.append(dataset_fold)

<class 'pandas.core.frame.DataFrame'>
Index: 2499150 entries, 0 to 2499999
Data columns (total 6 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   tweet                         object
 1   label                         int64 
 2   clean_tweet                   object
 3   tokenized_tweet               object
 4   tokenized_tweet_no_stopwords  object
 5   text                          object
dtypes: int64(1), object(5)
memory usage: 133.5+ MB
None


In [5]:
#load dataset from DatasetDict
dataset = folds_datasets[0]


In [20]:
MAX_LEN=512
from transformers import AutoTokenizer

hugging_face_model = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(hugging_face_model)


def tokenize_function(examples):
    return tokenizer(examples["clean_tweet"], max_length=MAX_LEN, padding='max_length',)


#convert labels to dtype float
dataset = dataset.map(lambda x: {"label": [float(x["label"])]})


In [21]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [22]:
tokenized_datasets['train'][0]['label']

[1.0]

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 19993
    })
    test: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 4998
    })
})

In [24]:
tokenized_datasets["train"] = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#tokenized_datasets["validation"] = tokenized_datasets["validation"].shuffle(seed=42)
tokenized_datasets["test"] = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [26]:
from transformers import DataCollatorWithPadding

tokenized_datasets["train"].set_format('torch', columns=["input_ids", "attention_mask", "label"] )
tokenized_datasets["test"].set_format('torch', columns=["input_ids", "attention_mask"] )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
import torch.nn as nn
from transformers import AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
class CustomModel(nn.Module):
    def __init__(self, checkpoint, num_labels ):
        super(CustomModel, self).__init__()
        self.num_labels = num_labels
        
        self.model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint, output_hidden_state = True ) )
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels )
        
    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask  )
        
        last_hidden_state = outputs[0]
        sequence_outputs = self.dropout(last_hidden_state)
        
        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))
        logits = torch.sigmoid(logits)
        loss = None
        if labels is not None:
            loss = torch.nn.functional.binary_cross_entropy(logits, labels)
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=last_hidden_state)

In [28]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32 
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle = True, batch_size = BATCH_SIZE, collate_fn = data_collator
)

#eval_dataloader = DataLoader(
#    tokenized_datasets['validation'], shuffle = True, batch_size = BATCH_SIZE, collate_fn = data_collator
#)

test_dataloader = DataLoader(
    tokenized_datasets['test'], batch_size = 32, collate_fn = data_collator
)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

model = CustomModel(checkpoint=hugging_face_model, num_labels=1).to(device)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
torch.cuda.get_device_name(device)

'NVIDIA GeForce GTX 1050'

In [31]:
for name, param in model.named_parameters():
  if "encoder.layer.0"  in name:
    param.requires_grad=False
  elif "encoder.layer.1."  in name:
    param.requires_grad=False
  elif "encoder.layer.2"  in name:
    param.requires_grad=False
  elif "encoder.layer.3"  in name:
    param.requires_grad=False
  elif "encoder.layer.4"  in name:
    param.requires_grad=False
  elif "encoder.layer.5"  in name:
    param.requires_grad=False
  elif "encoder.layer.6"  in name:
    param.requires_grad=False
  elif "encoder.layer.7"  in name:
    param.requires_grad=False
  elif "encoder.layer.8"  in name:
    param.requires_grad=False
  elif "encoder.layer.9"  in name:
    param.requires_grad=False
  elif "encoder.layer.10"  in name:
    param.requires_grad=False
  elif "encoder.layer.11"  in name:
    param.requires_grad=False
  elif "embeddings"  in name:
    param.requires_grad=False
  else:
    print(name)

model.pooler.dense.weight
model.pooler.dense.bias
classifier.weight
classifier.bias


In [32]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr = 5e-5 )

num_epoch = 1

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps=0,
    num_training_steps = num_training_steps,
    
)

/home/fgirlanda/CIL_sentiment_analysis/venv/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))

for epoch in range(num_epoch):
    model.train()
    for batch in train_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

  0%|          | 0/32 [01:25<?, ?it/s]
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
threshold = 0.5
correct_predictions = 0
total_predictions = 0

model.eval()
progress_bar_test = tqdm(range(num_epoch * len(test_dataloader) ))

metric_count = 0
for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad():
        outputs = model(**batch)
        
    logits = outputs.logits
    metric_count += torch.nn.functional.binary_cross_entropy(logits, batch['labels'])
    
    predictions = (logits >= threshold).int()  
    correct_predictions += (predictions == batch['labels']).sum().item()
    total_predictions += len(batch['labels'])

    progress_bar_test.update(1)
print("TEST ACC: ", correct_predictions / total_predictions)
print("TEST BCE: ", metric_count / len(test_dataloader))

  0%|          | 0/15620 [00:00<?, ?it/s]

TEST ACC:  0.5000260088430066
TEST BCE:  tensor(0.7389, device='cuda:0')


In [ ]:
torch.save(model.state_dict(), "roBERTa_12_layers_with_emb_light_pre")